In [1]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

sns.set_context("paper", font_scale = 1.4)

from plotly import express as exp, graph_objects as go, io as pio

pio.templates.default == 'ggplot2'

from plotly.subplots import make_subplots

import pickle

import datetime as dt

from pandas_utils.pandas_utils_2 import *


import json


import ipywidgets

from IPython.display import display

import warnings


warnings.filterwarnings("ignore", category=DeprecationWarning)

warnings.filterwarnings("ignore", category=FutureWarning)
    

In [2]:
years = [str(i) for i in range(2016, 2020, 1)]

In [3]:
domestic_visitors_loc = "./input_files/domestic_visitors/domestic_visitors_"

foreign_visitors_loc = "./input_files/foreign_visitors/foreign_visitors_"


dom_df = pd.DataFrame()


for_df = pd.DataFrame()



for y in years:
    
    
    dom_df = pd.concat([dom_df, pd.read_csv(domestic_visitors_loc + y + ".csv")], axis = 0)

    
    for_df = pd.concat([for_df, pd.read_csv(foreign_visitors_loc + y + ".csv")], axis = 0)
    


In [4]:
dom_df.shape, for_df.shape

((1512, 5), (1512, 5))

In [5]:
pd.DataFrame(dom_df.dtypes)

,0
district,object
date,object
month,object
year,int64
visitors,object


In [6]:
pd.DataFrame(for_df.dtypes)

,0
district,object
date,object
month,object
year,int64
visitors,object


In [7]:
dom_df.head(2)

,district,date,month,year,visitors
0,Adilabad,01-01-2016,January,2016,792136
1,Adilabad,01-02-2016,February,2016,937820


In [8]:

def get_quarter(date_vals):
    
    
    quarts = []
    
    
    for d in date_vals:
        
        
        
        if int(dt.datetime.strftime(d, "%m")) <= 3:
            
            
            quarts.append("Q1")
            
        
        elif int(dt.datetime.strftime(d, "%m")) > 3 and int(dt.datetime.strftime(d, "%m")) <= 6:
            
            
            quarts.append("Q2")
            
        
        
        elif int(dt.datetime.strftime(d, "%m")) > 6 and int(dt.datetime.strftime(d, "%m")) <= 9:
            
            
            
            quarts.append("Q3")
            
            
        
        
        elif int(dt.datetime.strftime(d, "%m")) > 9 and int(dt.datetime.strftime(d, "%m")) <= 12:
            
            
            quarts.append("Q4")
            
        
        
        else:
            
            
            quarts.append(np.nan)
    
    
    
    return quarts
            
            
            
        
        
        
    
    


def clean_df(df):
    
    
#     df['date'] = df['date'].apply(lambda x: dt.datetime.strptime(x, "%d-%m-%Y"))
    
    
    df['month_year'] = df['month'] +  ' ' + df['year'].apply(lambda x: str(x))
    
    
    df['date'] = df['month_year'].apply(lambda x: dt.datetime.strptime(x, "%B %Y"))
    
    
#     df['month_year'] = df['month_year'].apply(lambda x: dt.datetime.strptime(x, "%B %Y"))
    
    
    df['quarter'] = get_quarter(df['date'].map(dt.datetime.date).values.tolist())
    
    
    df['year_quarter'] = df['year'].apply(lambda x: str(x)) + '-' + df['quarter'] 
    
    
    df['visitors'] = df.visitors.replace(' ', 'nan')
    
    
#     try:
        
#         df['visitors'] = df.visitors.apply(lambda x: int(x) if x != ' ' else 0)
        
    
#     except BaseException as e:
        
        
    df['visitors'] = df.visitors.apply(lambda x: int(x) if str(x).strip().lower() != 'nan' else 0)
        
        
#         print(f"\n\n{e}\n\n")
        
    
#     else:
        
#         pass
    
    
    
    df = df[['district', 'date', 'month', 'year', 'month_year', 'quarter', 'year_quarter', 'visitors']]
    
    
    return df

In [9]:
dom_df = clean_df(dom_df)

for_df = clean_df(for_df)

In [10]:
dom_df.sample(3)

,district,date,month,year,month_year,quarter,year_quarter,visitors
211,Nalgonda,2016-08-01,August,2016,August 2016,Q3,2016-Q3,988102
298,Ranga Reddy,2019-11-01,November,2019,November 2019,Q4,2019-Q4,0
308,Suryapet,2017-09-01,September,2017,September 2017,Q3,2017-Q3,0


In [11]:
dom_df = dom_df.sort_values(by = ['district', 'date'], ascending = [True, True])

for_df = for_df.sort_values(by = ['district', 'date'], ascending = [True, True])

In [12]:
dom_df['foreign_visitors'] = for_df.visitors

In [13]:
dom_df = dom_df.rename({"visitors": 'domestic_visitors'}, axis = 1)

In [14]:
dom_df['d_to_f_ratio'] = dom_df.domestic_visitors / dom_df.foreign_visitors

In [15]:
# dom_df.d_to_f_ratio = dom_df.d_to_f_ratio.replace(np.inf, 0)

In [16]:
dom_df.head(5)

,district,date,month,year,month_year,quarter,year_quarter,domestic_visitors,foreign_visitors,d_to_f_ratio
0,Adilabad,2016-01-01,January,2016,January 2016,Q1,2016-Q1,792136,2,396068.0
1,Adilabad,2016-02-01,February,2016,February 2016,Q1,2016-Q1,937820,0,inf
2,Adilabad,2016-03-01,March,2016,March 2016,Q1,2016-Q1,582946,2,291473.0
3,Adilabad,2016-04-01,April,2016,April 2016,Q2,2016-Q2,341948,0,inf
4,Adilabad,2016-05-01,May,2016,May 2016,Q2,2016-Q2,252887,0,inf


In [17]:


district_filter = ['All']
    
    
unique_districts = list(dom_df.district.unique())
    
    
unique_districts.sort()
    
    
district_filter.extend(unique_districts)
    

time_axes = ['Month', 'year_quarter']




def plot_stats(visitor_type = 'Both', district_filter = ['All'], time_axis = ['Month']):
        
        
    
    df_ = dom_df
    
    
    
    if 'All' in district_filter:
        
        
        district_filter = list(df_.district.unique())
        
    
    else:
        
        
        district_filter = district_filter
        
    
    
    
    district_filter.sort()
        
    
    
    
    fig_df = df_[df_.district.isin(district_filter)]
    
    
    
    
    if visitor_type == 'Domestic and Foreign':
        

        
        
        if time_axis == 'Month':
            

            fig_df = fig_df.groupby(

                ['date'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

                "foreign_visitors": pd.Series.sum, 

                 "domestic_visitors": pd.Series.sum

                }


            )
            
            
            fig = make_subplots(specs=[[{"secondary_y": True}]])


#             fig = go.Figure()


            fig.add_trace(


                go.Scatter(


                    x = fig_df['date'],


                    y = fig_df['domestic_visitors'],


                    mode = 'lines+markers',


                    name = 'Domestic'


                    ),
                
                
                
                secondary_y = False




            )



            fig.add_trace(
                

                go.Scatter(


                    x = fig_df['date'],


                    y = fig_df['foreign_visitors'],


                    mode = 'lines+markers',


                    name = 'Foreign'




                ),
                
                
                
                secondary_y = True



            )
        
        
        
        else:
            
            
            fig_df = fig_df.groupby(

                ['year_quarter'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

                "foreign_visitors": pd.Series.sum, 

                 "domestic_visitors": pd.Series.sum

                }


            )

            
            
#             fig = go.Figure()


            fig = make_subplots(specs=[[{"secondary_y": True}]])
    


            fig.add_trace(


                go.Scatter(


                    x = fig_df['year_quarter'],


                    y = fig_df['domestic_visitors'],


                    mode = 'lines+markers',


                    name = 'Domestic'


                    ),
                
                
                
                secondary_y = False




            )



            fig.add_trace(

                go.Scatter(


                    x = fig_df['year_quarter'],


                    y = fig_df['foreign_visitors'],


                    mode = 'lines+markers',


                    name = 'Foreign'




                ),
                
                
                secondary_y = True



            )

        
    
        
        
        fig.update_yaxes(title = "Domestic Visitors", secondary_y = False)
        
        
        
        fig.update_yaxes(title = "Foreign Visitors", secondary_y = True)
        
        
        
        fig.update_yaxes(secondary_y=True, showgrid=False)
        
        
        
        hovertemp = "<br><br>".join([
            
            "<b>%{x}</b>", 
            
            "<b>%{y:.2s} visitors</b><extra></extra>"
        
        
        ])
        
        
        fig.update_traces(hovertemplate = hovertemp)
        
        
        
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic vs. Foreign Visitors for All Districts"
                
                )
            
            )
        
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic vs. Foreign Visitors for {district_filter[0]} District"
                
                )
            
            )
            
        
        
#         fig.update_layout(width = 780)
        
        
        fig.show()
            
            



        
    
    elif visitor_type == 'Domestic':
        
        
        
        if time_axis == 'Month':
            
            
        

            fig_df = fig_df.groupby(

                ['date'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

    #             "foreign_visitors": pd.Series.sum, 

                 "domestic_visitors": pd.Series.sum

                }


            )


            fig = exp.line(

                fig_df, 

                x='date', 

                y='domestic_visitors', 

                labels={'x':'date', 'y':'Domestic Visitors'},

                markers = True,

                title=''


            )
        
        
        
        else:
            
            
                    
            fig_df = fig_df.groupby(

                ['year_quarter'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

    #             "foreign_visitors": pd.Series.sum, 

                 "domestic_visitors": pd.Series.sum

                }


            )


            fig = exp.line(

                fig_df, 

                x='year_quarter', 

                y='domestic_visitors', 

                labels={'x':'date', 'y':'Domestic Visitors'},

                markers = True,

                title=''


            )
            
            
        
        
        
        hovertemp = "<br><br>".join([
            
            
            "<b>%{x}</b>", 
            
            
            "<b>%{y:.2s} visitors</b><extra></extra>"
        
        
        ])
        
        
        fig.update_traces(hovertemplate = hovertemp)


            
            
        
        
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic vs. Foreign Visitors for All Districts"
                
                )
            
            )
            
        
            
        
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic vs. Foreign Visitors for {district_filter[0]} District"
                
                )
            
            )
            
            
        
#         fig.update_layout(width = 780)
            
        
        
        
        fig.show()
            
    
        
        
    
    
    elif visitor_type == 'Foreign':
        
        
        if time_axis == 'Month':
        
        
            fig_df = fig_df.groupby(

                ['date'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

                "foreign_visitors": pd.Series.sum

    #              "domestic_visitors": pd.Series.sum

                }


            )



            fig = exp.line(

                fig_df, 

                x='date', 

                y='foreign_visitors', 

                labels={'x':'date', 'y':'Foreign Visitors'},

                markers = True,

                title=''


            )

        
        
        else:
            
            
            
            fig_df = fig_df.groupby(

                ['year_quarter'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

                "foreign_visitors": pd.Series.sum

    #              "domestic_visitors": pd.Series.sum

                }


            )



            fig = exp.line(

                fig_df, 

                x='year_quarter', 

                y='foreign_visitors', 

                labels={'x':'date', 'y':'Foreign Visitors'},

                markers = True,

                title=''


            )
            
        
        
                
        hovertemp = "<br><br>".join([
            
            
            "<b>%{x}</b>", 
            
            
            "<b>%{y:.2s} visitors</b><extra></extra>"
        
        
        ])
        
        
        fig.update_traces(hovertemplate = hovertemp)

            
            
        
        
        
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic vs. Foreign Visitors for All Districts"
                
                )
            
            )
        
        
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic vs. Foreign Visitors for {district_filter[0]} District"
                
                )
            
            )
            
            
            
            
        
#         fig.update_layout(width = 780)
            
            
        
        
        fig.show()
        
    
    
    
    else:
        
        
        
        if time_axis == 'Month':
        
        
            fig_df = fig_df.groupby(

                ['date'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

                "d_to_f_ratio": pd.Series.mean

    #              "domestic_visitors": pd.Series.sum

                }


            )
            
            
            
            fig_df['d_to_f_ratio'] = fig_df['d_to_f_ratio'].replace(np.inf, 0)

            


            fig = exp.line(

                fig_df, 

                x='date', 

                y="d_to_f_ratio", 

                labels={'x':'date', 'y':'d_to_f_ratio'},

                markers = True,

                title=''


            )

        
        
        else:
            
            
            
            fig_df = fig_df.groupby(

                ['year_quarter'], 

                as_index = False, 

                dropna = False

            ).agg(

                {

                "d_to_f_ratio": pd.Series.mean

    #              "domestic_visitors": pd.Series.sum

                }


            )
            
            
            
            fig_df['d_to_f_ratio'] = fig_df['d_to_f_ratio'].replace(np.inf, 0)



            fig = exp.line(

                fig_df, 

                x='year_quarter', 

                y='d_to_f_ratio', 

                labels={'x':'date', 'y':'d_to_f_ratio'},

                markers = True,

                title=''


            )
            
        
        
        
                
        hovertemp = "<br><br>".join([
            
            
            "<b>%{x}</b>", 
            
            
            "<b>D to F Ratio: %{y:.0f}</b><extra></extra>"
        
        
        ])
        
        
        fig.update_traces(hovertemplate = hovertemp)

            
            
        
        
        
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic to Foreign Ratio for All Districts"
                
                )
            
            )
        
        
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"Domestic to Foreign Ratio for {district_filter[0]} District"
                
                )
            
            )
            
            
            
            
        
#         fig.update_layout(width = 780)
            
            
        
        
        fig.show()
        
        
        
            
            
    
    
    
    

# ipywidgets.interact(
    
#     plot_stats, 
    
#     district_filter = district_filter, 
    
#     visitor_type = ['Domestic', 'Foreign', 'Domestic and Foreign', 'Domestic to Foreign Ratio'],
    
#     time_axis = time_axes

# )
    

In [18]:
# Get previous figures by 'All' or individual district

# Time Axes: date (month year) & Ouarter Year

In [19]:
# Calc Func


def percent_change_from_previous(df, col = 'month', metric = 'domestic_visitors', district_filter = []):
    
    
    time_val = ''
    
    
    if col == 'month':
        
        
        
        time_val = 'date'
        
    
    
    else:
        
        
        time_val = 'year_quarter'
        
        
        df['date'] = df.year_quarter.apply(lambda x: dom_df[dom_df.year_quarter == x]['date'].min())
        
        
        
    
    
    if 'list' not in str(type(metric)).lower():
        
    
    
        df[f'prev_{metric}'] = df[metric].shift(1)


        df[f'{metric}_percent_change'] = (

            (df[metric] - df[f'prev_{metric}']) / df[f'prev_{metric}']

        ) * 100
        
        
        
        
        
        fig = exp.line(

                df, 

                x=time_val, 

                y=f'{metric}_percent_change', 

                labels={'x':time_val, 'y':f'{metric}_percent_change'},

                markers = True,

                title=''


            )
        
        
    
        
        
        hovertemp = "<br><br>".join([
            
            
            "<b>%{x}</b>", 
            
            
            "<b>%{y:.2s}</b>" + f" <b>{metric.replace('_', ' ')}</b><extra></extra>"
        
        
        ])
        
        
        fig.update_traces(hovertemplate = hovertemp)


            
            
        
        
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric.replace('_', ' ').title()} Percent Change From Previous {col.title()} For All Districts"
                
                )
            
            )
            
        
            
        
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric.replace('_', ' ').title()} Percent Change From Previous {col.title()} For {district_filter[0]} District"
                
                )
            
            )
            
            
        
#         fig.update_layout(width = 780)
            
        
        
        
        fig.show()
            
        
        
        
        
        
        
        
    
    
    else:
        
        
        
        for m in metric:
            
                
            df[f'prev_{m}'] = df[m].shift(1)


            df[f'{m}_percent_change'] = (

                (df[m] - df[f'prev_{m}']) / df[f'prev_{m}']

            ) * 100
            
            
            
            
            fig = make_subplots(specs=[[{"secondary_y": True}]])




            fig.add_trace(


                go.Scatter(


                    x = df[time_val],


                    y = df[f'{metric[0]}_percent_change'],


                    mode = 'lines+markers',


                    name = 'Domestic'


                    ),
                
                
                
                secondary_y = False




            )



            fig.add_trace(
                

                go.Scatter(


                    x = df[time_val],


                    y = df[f'{metric[1]}_percent_change'],


                    mode = 'lines+markers',


                    name = 'Foreign'




                ),
                
                
                
                secondary_y = True



            )
            
            
            
        fig.update_yaxes(title = "Domestic Visitors", secondary_y = False, showgrid = True)
        
        
        
        fig.update_yaxes(title = "Foreign Visitors", secondary_y = True, showgrid = False)
        
        
        
        
        hovertemp = "<br><br>".join([
            
            "<b>%{x}</b>", 
            
            "<b>%{y:.2s} visitors</b><extra></extra>"
        
        
        ])
        
        
        fig.update_traces(hovertemplate = hovertemp)
        
        
        
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} Percent Change From Previous {col.title()} For All Districts"
                
                )
            
            )
            
        
            
        
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} Percent Change From Previous {col.title()} For {district_filter[0]} District"
                
                )
            
            )
        
        
        fig.show()
        
        
        
    
    
    

In [20]:
# Calc Func


def yoy_calc(df, col = 'month', metric = 'domestic_visitors', district_filter = []):
    
    
    time_val = ''
    
    
    
    if col == 'month':
        
        
        time_val = 'date'
        
    
    
    else:
        
        time_val = 'year_quarter'
        
        
        df['date'] = df.year_quarter.apply(lambda x: dom_df[dom_df.year_quarter == x]['date'].min())
        
        
        
    
    
    
    if 'list' not in str(type(metric)).lower():
        
        
        
        df['prev_year_date'] = df['date'].apply(lambda x: x - dt.timedelta(365))
        
        
        
        prev_year_dates = df.prev_year_date.values
        
        
        prev_year_vals = []
        
        
        for d in prev_year_dates:
    
    
            vals = df[df['date'] == d][metric].values
        
        
            if len(vals) == 0:
                
                
                prev_year_vals.append(np.nan)
                
            
            else:
                
                
                prev_year_vals.append(vals[0])
                
        
        
        
        df[f'prev_year_{metric}'] = prev_year_vals
        
        
        
        
        df[f'{metric}_prev_year_percent_change'] = (
            
            
            (df[metric] - df[f'prev_year_{metric}']) / df[f'prev_year_{metric}']
        
        
        ) * 100
        
        
        
        
        
                
        fig = exp.line(

                df, 

                x=time_val, 

                y=f'{metric}_prev_year_percent_change', 

                labels={'x':time_val, 'y':f'{metric}_prev_year_percent_change'},

                markers = True,

                title=''


            )
        
        
    
        
        
        hovertemp = "<br><br>".join([
            
            
            "<b>%{x}</b>", 
            
            
            "<b>%{y:.2f} %</b>" + " <b>vs. Previous Year</b><extra></extra>"
        
        
        ])
        
        
        
        fig.update_traces(hovertemplate = hovertemp)


            
            
        
        
        
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric.replace('_', ' ').title()} YOY Percent Change For All Districts"
                
                )
            
            )
            
        
            
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric.replace('_', ' ').title()} YOY Percent Change For {district_filter[0]} District"
                
                )
            
            )
            
            
        
#         fig.update_layout(width = 780)
            
        
        
        
        fig.show()
        
        
        
        
        
        
    
    
    else:
        
        
        
        df['prev_year_date'] = df['date'].apply(lambda x: x - dt.timedelta(365))
        
        
        
        prev_year_dates = df.prev_year_date.values
        
        
        
        
        # Dual Axis
        
        
        for m in metric:
            
            
            
            prev_year_vals = []

            

            for d in prev_year_dates:


                vals = df[df['date'] == d][m].values


                if len(vals) == 0:


                    prev_year_vals.append(np.nan)


                else:


                    prev_year_vals.append(vals[0])




            df[f'prev_year_{m}'] = prev_year_vals
            
            

            df[f'{m}_prev_year_percent_change'] = (


                (df[m] - df[f'prev_year_{m}']) / df[f'prev_year_{m}']


            ) * 100
            
            
            
            # Dual Axis Chart
            
            
            
            fig = make_subplots(specs=[[{"secondary_y": True}]])




            fig.add_trace(


                go.Scatter(


                    x = df[time_val],


                    y = df[f'{metric[0]}_prev_year_percent_change'],


                    mode = 'lines+markers',


                    name = 'Domestic'


                    ),
                
                
                
                secondary_y = False




            )



            fig.add_trace(
                

                go.Scatter(


                    x = df[time_val],


                    y = df[f'{metric[1]}_prev_year_percent_change'],


                    mode = 'lines+markers',


                    name = 'Foreign'




                ),
                
                
                
                secondary_y = True



            )
            
            
            
        fig.update_yaxes(title = f"{metric[0].replace('_', ' ').title()} YOY % Change", secondary_y = False, showgrid = True)
        
        
        
        fig.update_yaxes(title = f"{metric[1].replace('_', ' ').title()} YOY % Change", secondary_y = True, showgrid = False)
        
        
        
        
        hovertemp = "<br><br>".join([
            
            "<b>%{x}</b>", 
            
            "<b>%{y:.2f} % vs. Previous Year</b><extra></extra>"
        
        
        ])
        
        
        
        fig.update_traces(hovertemplate = hovertemp)
        
        
        
        
                
        if len(district_filter) == 33:
        
        
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} YOY Percent Change For All Districts"
                
                )
            
            )
            
        
            
        
        
        else:
            
                   
            fig.update_layout(
                
                title = dict(
                    
                    text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} YOY Percent Change For {district_filter[0]} District"
                
                )
            
            )
            
        
        
        
        
        fig.show()
    
    
    

In [21]:
def ytm_ytq_calc(df, col = 'month', metric = 'domestic_visitors', district_filter = []):
    
   

    time_val = ''
    
    
    
    
    if col == 'month':
        
        
        time_val = 'date'
        
    
    
    else:
        
        time_val = 'year_quarter'
        
        
        df['date'] = df.year_quarter.apply(lambda x: dom_df[dom_df.year_quarter == x]['date'].min())
        
        
    
    
    df['first_date_year'] = df['date'].apply(
        
        lambda x: x - dt.timedelta(int(dt.datetime.strftime(x, "%j")) - 1)
    
    
    )
    
    
        
    
    
    
    if 'list' not in str(type(metric)).lower():
        
        
        
        
        first_date_year_dates = df.first_date_year.values
        
        
        first_date_year_vals = []
        
        
        for d in first_date_year_dates:
    
    
            vals = df[df['date'] == d][metric].values
        
        
            if len(vals) == 0:
                
                
                first_date_year_vals.append(np.nan)
                
            
            else:
                
                
                first_date_year_vals.append(vals[0])
                
        
        
        
        df[f'first_date_year_{metric}'] = first_date_year_vals
        
        
        
        
        df[f'{metric}_first_date_year_percent_change'] = (
            
            
            (df[metric] - df[f'first_date_year_{metric}']) / df[f'first_date_year_{metric}']
        
        
        ) * 100
        
        
        
        
        
                
        fig = exp.line(

                df, 

                x=time_val, 

                y=f'{metric}_first_date_year_percent_change', 

                labels={'x':time_val, 'y':f'{metric}_first_date_year_percent_change'},

                markers = True,

                title=''


            )
        
        
        
        
        if time_val == 'date':
        
        
            hovertemp = "<br><br>".join([


                "<b>%{x}</b>", 


                " <b>YTM: </b>" + "<b>%{y:.2f} %</b><extra></extra>"


            ])
        
        
        
        else:
            
            
                    
            hovertemp = "<br><br>".join([


                "<b>%{x}</b>", 


                " <b>YTM: </b>" + "<b>%{y:.2f} %</b><extra></extra>"


            ])
        
        
        
        fig.update_traces(hovertemplate = hovertemp)


            
            
        if time_val == 'date':
            
            
        
        
            if len(district_filter) == 33:


                fig.update_layout(

                    title = dict(

                        text = f"{metric.replace('_', ' ').title()} YTM Percent Change For All Districts"

                    )

                )




            else:


                fig.update_layout(

                    title = dict(

                        text = f"{metric.replace('_', ' ').title()} YTM Percent Change For {district_filter[0]} District"

                    )

                )

        
        
        
        
        else:
            
            
            
            if len(district_filter) == 33:


                fig.update_layout(

                    title = dict(

                        text = f"{metric.replace('_', ' ').title()} YTQ Percent Change For All Districts"

                    )

                )




            else:


                fig.update_layout(

                    title = dict(

                        text = f"{metric.replace('_', ' ').title()} YTQ Percent Change For {district_filter[0]} District"

                    )

                )

            
            
            
            
            
            
        
#         fig.update_layout(width = 780)
            
        
        
        
        fig.show()
        
        
        
        
        
        
    
    
    else:
        
        
        
        
        # Dual Axis
        
        
        for m in metric:
            
            
            
            first_date_year_dates = df.first_date_year.values
        

            first_date_year_vals = []


            for d in first_date_year_dates:


                vals = df[df['date'] == d][m].values


                if len(vals) == 0:


                    first_date_year_vals.append(np.nan)


                else:


                    first_date_year_vals.append(vals[0])




            df[f'first_date_year_{m}'] = first_date_year_vals




            df[f'{m}_first_date_year_percent_change'] = (


                (df[m] - df[f'first_date_year_{m}']) / df[f'first_date_year_{m}']


            ) * 100


            
            # Dual Axis Chart
            
            
            
            fig = make_subplots(specs=[[{"secondary_y": True}]])




            fig.add_trace(


                go.Scatter(


                    x = df[time_val],


                    y = df[f'{metric[0]}_first_date_year_percent_change'],


                    mode = 'lines+markers',


                    name = 'Domestic'


                    ),
                
                
                
                secondary_y = False




            )



            fig.add_trace(
                

                go.Scatter(


                    x = df[time_val],


                    y = df[f'{metric[1]}_first_date_year_percent_change'],


                    mode = 'lines+markers',


                    name = 'Foreign'




                ),
                
                
                
                secondary_y = True



            )
            
            
        if time_val == 'date':
            
            
            
            fig.update_yaxes(title = f"{metric[0].replace('_', ' ').title()} YTM % Change", secondary_y = False, showgrid = True)



            fig.update_yaxes(title = f"{metric[1].replace('_', ' ').title()} YTM % Change", secondary_y = True, showgrid = False)

            
            
            hovertemp = "<br><br>".join([
            
                "<b>%{x}</b>", 

                "<b>YTM: </b>" + "<b>%{y:.2f} %</b><extra></extra>"
        
        
            ])
            
            

        else:
        
        
             fig.update_yaxes(title = f"{metric[0].replace('_', ' ').title()} YTM % Change", secondary_y = False, showgrid = True)
        
        
        
             fig.update_yaxes(title = f"{metric[1].replace('_', ' ').title()} YTM % Change", secondary_y = True, showgrid = False)
        
             
            
             hovertemp = "<br><br>".join([
            
                "<b>%{x}</b>", 

                "<b>YTQ: </b>" + "<b>%{y:.2f} %</b><extra></extra>"
        
        
             ])
        
        

        
        
        
        fig.update_traces(hovertemplate = hovertemp)
        
        
        
        
        if time_val == 'date':
            
            

            if len(district_filter) == 33:


                fig.update_layout(

                    title = dict(

                        text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} YTM Percent Change For All Districts"

                    )

                )





            else:


                fig.update_layout(

                    title = dict(

                        text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} YTM Percent Change For {district_filter[0]} District"

                    )

                )

    
            
            
        
        
        else:
            
        
        
        
        
                
            if len(district_filter) == 33:


                fig.update_layout(

                    title = dict(

                        text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} YTQ Percent Change For All Districts"

                    )

                )





            else:


                fig.update_layout(

                    title = dict(

                        text = f"{metric[0].replace('_', ' ').title()} vs. {metric[1].replace('_', ' ').title()} YTQ Percent Change For {district_filter[0]} District"

                    )

                )



                
        
        fig.show()
    
    
    
    

In [22]:

district_filter = ['All']



unique_districts = list(dom_df.district.unique())



unique_districts.sort()



district_filter.extend(unique_districts)



time_axes = ['Month', 'Quarter']



metrics = [
    
    'Domestic Visitors', 
    
    'Foreign Visitors', 
    
    'Domestic and Foreign Visitors', 
    
    'Domestic to Foreign Visitor Ratio'

]



calcs = {

        "Month": [

            'Percent Change From Previous Month', 

            'YOY', 

            'YTM'

        ],


        "Quarter": [

            "Percent Change From Previous Quarter", 

            'YOY', 

            'YTQ'

        ]

    }



# global time_axis


# time_axis = 'Month'



def more_calcs(
    
    
    metric = metrics[0], 
    
    
    time_axis = time_axes[0], 
    
    
    calc = calcs['Month'][0],
    
    
    district_filter = 'All'

):
    
    

    if district_filter == 'All':

        

        district_filter = unique_districts



    else:


        district_filter = district_filter




    df_ = dom_df



    df_ = df_[df_.district.isin(district_filter)]




    if time_axis == 'Month':



        if metric == 'Domestic Visitors':



            df_ = df_.groupby(

                ['date'], 

                dropna = False, 

                as_index = False

            ).agg(


                {"domestic_visitors": pd.Series.sum}

            )


            if calc == 'Percent Change From Previous Month':



                percent_change_from_previous(

                    df = df_, 


                    col = 'month', 


                    metric = 'domestic_visitors', 


                    district_filter = district_filter


                )




            elif calc == 'YOY':


                yoy_calc(

                    df = df_, 

                    col = 'month', 

                    metric = 'domestic_visitors', 

                    district_filter = district_filter

                )



            elif calc == 'YTM':


                ytm_ytq_calc(

                    df = df_, 

                    col = 'month', 

                    metric = 'domestic_visitors', 

                    district_filter = district_filter

                )



            else:

                pass





        elif metric == 'Foreign Visitors':



            df_ = df_.groupby(

                ['date'], 

                dropna = False, 

                as_index = False

            ).agg(


                {"foreign_visitors": pd.Series.sum}

            )



            if calc == 'Percent Change From Previous Month':



                percent_change_from_previous(

                    df = df_, 


                    col = 'month', 


                    metric = 'foreign_visitors', 


                    district_filter = district_filter


                )




            elif calc == 'YOY':


                yoy_calc(

                    df = df_, 

                    col = 'month', 

                    metric = 'foreign_visitors', 

                    district_filter = district_filter


                )



            elif calc == 'YTM':


                ytm_ytq_calc(

                    df = df_, 

                    col = 'month', 

                    metric = 'foreign_visitors', 

                    district_filter = district_filter

                )



            else:

                pass




        elif metric == 'Domestic and Foreign Visitors':



            df_ = df_.groupby(

                ['date'], 

                dropna = False, 

                as_index = False

            ).agg(


                {

                    "domestic_visitors": pd.Series.sum, 


                    "foreign_visitors": pd.Series.sum

                }

            )



            if calc == 'Percent Change From Previous Month':



                percent_change_from_previous(

                    df = df_, 


                    col = 'month', 


                    metric = ['domestic_visitors', 'foreign_visitors'], 


                    district_filter = district_filter


                )




            elif calc == 'YOY':


                yoy_calc(

                    df = df_, 

                    col = 'month', 

                    metric = ['domestic_visitors', 'foreign_visitors'], 

                    district_filter = district_filter

                )



            elif calc == 'YTM':



                ytm_ytq_calc(

                    df = df_, 

                    col = 'month', 

                    metric = ['domestic_visitors', 'foreign_visitors'], 

                    district_filter = district_filter

                )



            else:

                pass




        elif metric == 'Domestic to Foreign Visitor Ratio':



            df_ = df_.groupby(

                ['date'], 

                dropna = False, 

                as_index = False

            ).agg(

                {"d_to_f_ratio": pd.Series.mean}

            )


            if calc == 'Percent Change From Previous Month':



                percent_change_from_previous(

                    df = df_, 


                    col = 'month', 


                    metric = 'd_to_f_ratio', 


                    district_filter = district_filter


                )




            elif calc == 'YOY':


                yoy_calc(

                    df = df_, 

                    col = 'month', 

                    metric = 'd_to_f_ratio', 

                    district_filter = district_filter

                )



            elif calc == 'YTM':


                ytm_ytq_calc(

                    df = df_, 

                    col = 'month', 

                    metric = 'd_to_f_ratio', 

                    district_filter = district_filter

                )



            else:

                pass



        else:


            pass






    else:


        if metric == 'Domestic Visitors':



            df_ = df_.groupby(

                ['year_quarter'], 

                dropna = False, 

                as_index = False

            ).agg(


                {"domestic_visitors": pd.Series.sum}

            )



            if calc == 'Percent Change From Previous Quarter':



                percent_change_from_previous(

                    df = df_, 


                    col = 'quarter', 


                    metric = 'domestic_visitors', 


                    district_filter = district_filter


                )




            elif calc == 'YOY':


                yoy_calc(

                    df = df_, 

                    col = 'quarter', 

                    metric = 'domestic_visitors', 

                    district_filter = district_filter


                )



            elif calc == 'YTQ':


                ytm_ytq_calc(

                    df = df_, 

                    col = 'quarter', 

                    metric = 'domestic_visitors', 

                    district_filter = district_filter

                )



            else:

                pass




        elif metric == 'Foreign Visitors':



            df_ = df_.groupby(

                ['year_quarter'], 

                dropna = False, 

                as_index = False

            ).agg(


                {"foreign_visitors": pd.Series.sum}

            )



            if calc == 'Percent Change From Previous Quarter':



                percent_change_from_previous(

                    df = df_, 


                    col = 'quarter', 


                    metric = 'foreign_visitors', 


                    district_filter = district_filter


                )




            elif calc == 'YOY':


                yoy_calc(

                    df = df_, 

                    col = 'quarter', 

                    metric = 'foreign_visitors', 

                    district_filter = district_filter


                )



            elif calc == 'YTQ':


                ytm_ytq_calc(

                    df = df_, 

                    col = 'quarter', 

                    metric = 'foreign_visitors', 

                    district_filter = district_filter

                )



            else:

                pass




        elif metric == 'Domestic and Foreign Visitors':



            df_ = df_.groupby(

                ['year_quarter'], 

                dropna = False, 

                as_index = False

            ).agg(


                {

                    "domestic_visitors": pd.Series.sum, 


                    "foreign_visitors": pd.Series.sum

                }

            )



            if calc == 'Percent Change From Previous Quarter':



                percent_change_from_previous(

                    df = df_, 


                    col = 'quarter', 


                    metric = ['domestic_visitors', 'foreign_visitors'], 


                    district_filter = district_filter


                )




            elif calc == 'YOY':


                yoy_calc(

                    df = df_, 

                    col = 'quarter', 

                    metric = ['domestic_visitors', 'foreign_visitors'], 

                    district_filter = district_filter


                )



            elif calc == 'YTQ':


                ytm_ytq_calc(

                    df = df_, 

                    col = 'quarter', 

                    metric = ['domestic_visitors', 'foreign_visitors'], 

                    district_filter = district_filter

                )



            else:

                pass




        elif metric == 'Domestic to Foreign Visitor Ratio':



            df_ = df_.groupby(

                ['year_quarter'], 

                dropna = False, 

                as_index = False

            ).agg(

                {"d_to_f_ratio": pd.Series.mean}

            )


            if calc == 'Percent Change From Previous Quarter':



                percent_change_from_previous(

                    df = df_, 


                    col = 'quarter', 


                    metric = 'd_to_f_ratio', 


                    district_filter = district_filter

                )




            elif calc == 'YOY':


                yoy_calc(


                    df = df_, 


                    col = 'quarter', 


                    metric = 'd_to_f_ratio', 


                    district_filter = district_filter


                )



            elif calc == 'YTQ':



                ytm_ytq_calc(

                    df = df_, 

                    col = 'quarter', 

                    metric = 'd_to_f_ratio', 

                    district_filter = district_filter

                )



            else:

                pass



        else:


            pass


        

# ipywidgets.interact(
    
#     more_calcs,     
    
    
#     metric = metrics, 
    
    
#     time_axis = time_axes, 
    
    
#     calc = calcs[time_axis], 
    
    
#     district_filter = district_filter


# )



In [23]:
# Importing telangana districts geojson

with open("./input_files/shape_files/TS_District_Boundary_33_FINAL.geojson", 'r') as gfile:
    
    gjson = gfile.read()

    
    
gjson = json.loads(gjson)

In [24]:
# dict_keys(['type', 'name', 'features'])

gjson['features'][0]

{'type': 'Feature',
 'properties': {'Shape_Leng': 397121.089982,
  'STATE': 'Telangana',
  'DISTRICT_N': 'Adilabad',
  'Shape_Le_1': 397121.089982,
  'Shape_Area': 3980340811.38},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[221272.66270000022, 2200734.384299999],
    [221272.68259999994, 2200734.3841999993],
    [221292.18659999967, 2200735.5699],
    [221353.16789999977, 2200739.276799999],
    [221399.39809999987, 2200742.0873],
    [221440.69230000023, 2200742.5864],
    [221549.33779999986, 2200743.9],
    [221565.1836000001, 2200740.241699999],
    [221600.00289999973, 2200732.2032999992],
    [221625.98819999956, 2200726.2040999997],
    [221659.61670000013, 2200717.2737],
    [221723.5307, 2200700.3007999994],
    [221776.8541000001, 2200693.270199999],
    [221895.471, 2200677.630999999],
    [221940.29069999978, 2200671.1371999998],
    [221940.3030000003, 2200671.1352999993],
    [222010.48010000028, 2200660.9674999993],
    [222022.79559999984, 2200660.9255],
    [2

In [28]:
_ = dom_df[
    
    dom_df['year'] == '2018'

].groupby(
    
    ['district'], 
    
    as_index = False, 
    
    dropna = False

).agg(
    
    
    
    {
        
        "domestic_visitors": pd.Series.sum, 
        
        "foreign_visitors": pd.Series.sum
    
    }


)

In [32]:
# import plotly.express as px

# df = px.data.election()
# geojson = px.data.election_geojson()

# fig = px.choropleth(df, geojson=geojson, color="Bergeron",
#                     locations="district", featureidkey="properties.district",
#                     projection="mercator"
#                    )
# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()


fig = exp.choropleth(
    
    _, 
    
    geojson = gjson, 
    
    
    color = 'domestic_visitors', 
    
    
    locations = 'district', 
    
    
    featureidkey = 'properties.DISTRICT_N',
    
    
    projection = 'domestic_visitors'


)


fig.update_geos(fitbounds = "locations", visible = False)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


fig.show()


ValueError: 
    Invalid value of type 'builtins.str' received for the 'type' property of layout.geo.projection
        Received value: 'domestic_visitors'

    The 'type' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['airy', 'aitoff', 'albers', 'albers usa', 'august',
            'azimuthal equal area', 'azimuthal equidistant', 'baker',
            'bertin1953', 'boggs', 'bonne', 'bottomley', 'bromley',
            'collignon', 'conic conformal', 'conic equal area', 'conic
            equidistant', 'craig', 'craster', 'cylindrical equal
            area', 'cylindrical stereographic', 'eckert1', 'eckert2',
            'eckert3', 'eckert4', 'eckert5', 'eckert6', 'eisenlohr',
            'equal earth', 'equirectangular', 'fahey', 'foucaut',
            'foucaut sinusoidal', 'ginzburg4', 'ginzburg5',
            'ginzburg6', 'ginzburg8', 'ginzburg9', 'gnomonic',
            'gringorten', 'gringorten quincuncial', 'guyou'

In [ ]:


def district_choropleth(dom_df, gjson):




    dom_grp_df = pd.DataFrame()






    if metric != 'Domestic and Foreign Visitors':


        dom_grp_df = dom_df.groupby(


            ['district'], 

            as_index = False, 

            dropna = False

        ).agg(


            {f"{'_'.join(metric.lower().split(' '))}": pd.Series.sum}


        )


    else:


        dom_grp_df = dom_grp_df.groupby(


            ['district'], 


            as_index = False, 


            dropna = False


        ).agg(

            {

                "domestic_visitors": pd.Series.sum, 


                "foreign_visitors": pd.Series.sum

            }


        )



        dom_grp_df['total_visitors'] = dom_grp_df.domestic_visitors + dom_grp_df.foreign_visitors






    exp.set_mapbox_access_token(st.secrets["mapbox_access_token"])



    if len(metric.split(' ')) > 2:


            fig = exp.choropleth_mapbox(


                dom_grp_df, 


                geojson = gjson, 


                color = "total_visitors",


                locations = 'district',


                featureidkey = 'properties.DISTRICT_N',


                color_continuous_scale = exp.colors.sequential.Oranges,


                center = dict(lat = 17.77472150260277, lon = 17.77472150260277),


                mapbox_style = "carto-positron",


                zoom = 7.0,



                custom_data = [


                    dom_grp_df.district, 


                    dom_grp_df["total_visitors"],


                    dom_grp_df['domestic_visitors'],


                    dom_grp_df['foreign_visitors']


                ]



        )




        hovertemp = "<br><br>".join(


                [

                    "<b>%{customdata[0]}</b>", 


                    "<b>Domestic Visitors: %{customdata[2]:.2s}</b>",


                    "<b>Foreign Visitors: %{customdata[3]:.2s}</b>",


                    "<b>Total Visitors: %{customdata[1]:.2s}</b><extra></extra>"


                ]


        )


        fig.update_traces(hovertemplate = hovertemp)









    else:


        fig = exp.choropleth_mapbox(


            dom_grp_df, 


            geojson = gjson, 


            color = f"{'_'.join(metric.lower().split(' '))}",


            locations = 'district',


            featureidkey = 'properties.DISTRICT_N',


            color_continuous_scale = exp.colors.sequential.Oranges,


            center = dict(lat = 17.77472150260277, lon = 17.77472150260277),


            mapbox_style = "carto-positron",


            zoom = 7.0,



            custom_data = [


                dom_grp_df.district, 


                dom_grp_df[f"{'_'.join(metric.lower().split(' '))}"]


            ]



        )





        hovertemp = "<br><br>".join(


                [

                    "<b>%{customdata[0]}</b>", 


                    f"<b>{metric}: </b>" + "<b>%{customdata[1]:.2s}</b><extra></extra>"


                ]


        )



        fig.update_traces(hovertemplate = hovertemp)




    fig.update_layout(



            margin = dict(r = 0, t = 0, l = 0, b = 0)


    )




    fig.update_layout(coloraxis_colorbar = dict(title = " "))





    st.plotly_chart(fig, use_container_width = True)



    
    

exp.set_mapbox_access_token(st.secrets["mapbox_access_token"])




	fig = exp.choropleth_mapbox(


		df_dist_grp,


		geojson = districts_geojson,


		color = "incidents",


		locations = 'rpt_dist_no',


		featureidkey = 'properties.REPDIST',


		color_continuous_scale = exp.colors.sequential.Oranges,


		center = dict(lat = df.lat.median(), lon = df.lon.median()),


		mapbox_style = "carto-positron",


		zoom = 8.7,


		custom_data = [



		df_dist_grp.rpt_dist_no, 


		df_dist_grp.area_name, 


		df_dist_grp.incidents.apply(lambda x: f'{x:,}')



		]


		)





	# fig.update_layout(

	# 	mapbox = dict(


	# 		layers = [


	# 		dict(


	# 			source = json.loads(div_gdf.geometry.to_json()),


	# 			# below = "traces",


	# 			type = "line",


	# 			color = "#36454F",


	# 			line = dict(width = 0.8)



	# 			)


	# 		]

	# 		)

	# 	)




	hovertemp = "<br><br>".join(

		[


		"Reporting District: <b>%{customdata[0]}</b>", 


		"Division Name: <b>%{customdata[1]}</b>",


		"No. of Incidents: <b>%{customdata[2]}</b>"



		]


		)


	fig.update_traces(hovertemplate = hovertemp)




	fig.update_layout(


		margin = dict(r = 0,t = 0, l = 0, b = 0)


		)



	fig.update_layout(coloraxis_colorbar = dict(title = "Incidents"))




	st.plotly_chart(fig, use_container_width = True)

,0
district,object
date,datetime64[ns]
month,object
year,int64
month_year,object
quarter,object
year_quarter,object
domestic_visitors,int64
foreign_visitors,int64
d_to_f_ratio,float64
